# CAMP model (capital asset pricing model) for stock market indices

**The necessary libraries are imported**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import yfinance as yf
from statsmodels.stats.stattools import durbin_watson

If we need to install the yfinance library we enter: *!pip install yfinance*

**For this project we will use stock prices from the last 5 years to today (03/05/2024)**

**We bring the data from Yahoo Finance**

In [2]:
djt = yf.Ticker("^IBEX")

In [3]:
djt.info

{'maxAge': 86400,
 'priceHint': 2,
 'previousClose': 10001.3,
 'open': 10015.0,
 'dayLow': 10012.9,
 'dayHigh': 10107.9,
 'regularMarketPreviousClose': 10001.3,
 'regularMarketOpen': 10015.0,
 'regularMarketDayLow': 10012.9,
 'regularMarketDayHigh': 10107.9,
 'volume': 172079653,
 'regularMarketVolume': 172079653,
 'averageVolume': 140433845,
 'averageVolume10days': 143261890,
 'averageDailyVolume10Day': 143261890,
 'bid': 10048.6,
 'ask': 10055.5,
 'fiftyTwoWeekLow': 8501.2,
 'fiftyTwoWeekHigh': 10301.3,
 'fiftyDayAverage': 10022.298,
 'twoHundredDayAverage': 9604.088,
 'currency': 'EUR',
 'exchange': 'MCE',
 'quoteType': 'INDEX',
 'symbol': '^IBEX',
 'underlyingSymbol': '^IBEX',
 'shortName': 'IBEX 35...',
 'longName': 'IBEX 35...',
 'firstTradeDateEpochUtc': 742460400,
 'timeZoneFullName': 'Europe/Madrid',
 'timeZoneShortName': 'CET',
 'uuid': '27b23394-5d7d-3536-8476-fa4905cef873',
 'messageBoardId': 'finmb_INDEXIBEX',
 'gmtOffSetMilliseconds': 3600000,
 'trailingPegRatio': None}

In [4]:
djt.history_metadata

{'currency': 'EUR',
 'symbol': '^IBEX',
 'exchangeName': 'MCE',
 'instrumentType': 'INDEX',
 'firstTradeDate': 742460400,
 'regularMarketTime': 1709311041,
 'hasPrePostMarketData': False,
 'gmtoffset': 3600,
 'timezone': 'CET',
 'exchangeTimezoneName': 'Europe/Madrid',
 'regularMarketPrice': 10064.7,
 'chartPreviousClose': 10130.6,
 'previousClose': 10001.3,
 'scale': 3,
 'priceHint': 2,
 'currentTradingPeriod': {'pre': {'timezone': 'CET',
   'start': 1709539200,
   'end': 1709539200,
   'gmtoffset': 3600},
  'regular': {'timezone': 'CET',
   'start': 1709539200,
   'end': 1709569800,
   'gmtoffset': 3600},
  'post': {'timezone': 'CET',
   'start': 1709569800,
   'end': 1709569800,
   'gmtoffset': 3600}},
 'tradingPeriods':                                           pre_start                   pre_end  \
 Date                                                                            
 2024-02-26 00:00:00+01:00 2024-02-23 09:00:00+01:00 2024-02-23 09:00:00+01:00   
 2024-02-27 00:00:00+

In [5]:
data = yf.Tickers('ANA.MC, ANE.MC, ACX.MC, ACS.MC, AENA.MC, AMS.MC,MTS.MC, SAB.MC, BKT.MC, BBVA.MC, CABK.MC, CLNX.MC, COL.MC, ENG.MC, ELE.MC, FER.MC, FDR.MC, GRF.MC, IAG.MC, IBE.MC, ITX.MC, IDR.MC, LOG.MC, MAP.MC, MEL.MC, MRL.MC, NTGY.MC, RED.MC, REP.MC, ROVI.MC, SCYR.MC, SAN.MC, SLR.MC, TEF.MC, UNI.MC, ^IBEX').history('5Y')['Close']
data = pd.DataFrame(data)
data.head()

[*********************100%%**********************]  36 of 36 completed


Ticker,ACS.MC,ACX.MC,AENA.MC,AMS.MC,ANA.MC,ANE.MC,BBVA.MC,BKT.MC,CABK.MC,CLNX.MC,...,RED.MC,REP.MC,ROVI.MC,SAB.MC,SAN.MC,SCYR.MC,SLR.MC,TEF.MC,UNI.MC,^IBEX
Date,,,,,,,,,,,,,,,,,,,,,
2019-03-01,27.440075,6.482483,135.164566,64.124420,77.760315,NaN,4.094505,3.882566,2.542245,18.269133,...,13.350863,11.007013,16.912813,0.823138,3.355284,1.732557,5.26,4.669833,0.901208,9267.700195
2019-03-04,27.619604,6.637560,136.377975,63.602924,77.413162,NaN,4.061772,3.873889,2.544702,19.029978,...,13.407871,10.996090,16.959024,0.822727,3.345031,1.748954,5.27,4.635491,0.895907,9259.799805
2019-03-05,27.308886,6.605427,138.068069,63.197323,78.020676,NaN,4.039697,3.904256,2.543883,19.415398,...,13.158456,10.985168,16.912813,0.821987,3.347003,1.756761,5.20,4.676702,0.899441,9258.200195
2019-03-06,27.274366,6.578884,137.634720,62.714455,78.090111,NaN,4.089175,3.880396,2.547159,19.407366,...,13.343735,10.996090,16.866606,0.833417,3.402603,1.780185,4.99,4.699803,0.892373,9296.700195
2019-03-07,27.163881,6.387482,137.374695,63.757431,78.836456,NaN,3.972712,3.762184,2.448877,19.375246,...,13.696481,10.970603,16.727976,0.772977,3.283123,1.744269,5.23,4.717911,0.874702,9249.900391


In [89]:
class Camp():
    "This class represents the CAMP model"
    def __init__(self, df, market, stock):
        self._df = df
        self._market = market
        self._stock = stock
    
    def log_returns(self):
        returns = np.log(self._df/self._df.shift(1))
        returns.dropna(inplace=True)
            
    
    def parameter

In [139]:
class Camp():
    "This class represents the CAMP model"
    def __init__(self, df, market, stock):
        self._df = df
        self._market = market
        self._stock = stock
    
    def df_model(self):
        df_returns = np.log(self._df/self._df.shift(1))
        df_returns.dropna(inplace=True)
        self.model = sm.OLS(df_returns[self._market], df_returns[self._stock]).fit()
        print(self.model.summary())

In [141]:
daily_return = Camp(data, "^IBEX", "BBVA.MC")
#daily_return.returns()

In [142]:
daily_return.df_model()

                                 OLS Regression Results                                
Dep. Variable:                  ^IBEX   R-squared (uncentered):                   0.586
Model:                            OLS   Adj. R-squared (uncentered):              0.585
Method:                 Least Squares   F-statistic:                              966.2
Date:                Mon, 04 Mar 2024   Prob (F-statistic):                   7.15e-133
Time:                        00:11:05   Log-Likelihood:                          2446.9
No. Observations:                 684   AIC:                                     -4892.
Df Residuals:                     683   BIC:                                     -4887.
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------